In [64]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk

# Load captions
captions_file = "C:/Users/ewang/OneDrive/Desktop/Spring_2024/COS_429/Final/captions_BLIP_cos_20_percent.csv"
captions = pd.read_csv(captions_file)
light_captions = captions["light_caption"]
dark_captions = captions["dark_caption"]

# # Load top 100 words
# top100_file = "C:/Users/ewang/OneDrive/Desktop/Spring_2024/COS_429/Final/imagecaptioning-bias/annotations/top100_overall.txt"
# with open(top100_file, 'r') as file:
#     top_100_words = file.read().splitlines()

nltk.download('stopwords')
stop_words = list(set(stopwords.words('english')))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ewang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:

# Create feature vectors for light and dark captions
# vectorizer = CountVectorizer(vocabulary=top_100_words, binary=True)
vectorizer = CountVectorizer(max_features=100, stop_words=stop_words)
all_captions = pd.concat([light_captions, dark_captions])
vectorizer.fit(all_captions)
X_light = vectorizer.transform(light_captions).toarray()
X_dark = vectorizer.transform(dark_captions).toarray()

# Combine features and create labels
X = pd.concat([pd.DataFrame(X_light), pd.DataFrame(X_dark)], ignore_index=True)
y = [0] * len(light_captions) + [1] * len(dark_captions)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [66]:
# Predict and evaluate on the test set
y_pred = model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_pred)
print(f"AUC Score: {auc_score}")

# Show the most important words based on the trained model
feature_names = vectorizer.get_feature_names_out()
feature_importance = model.coef_[0]
important_words = sorted(zip(feature_importance, feature_names), key=lambda x: x[0], reverse=True)
print("Top 10 words for Dark skin tone captions:")
for coef, word in important_words[:10]:
    print(f"{word}: {coef}")
print("Top 10 words for Light skin tone captions:")
for coef, word in important_words[-10:]:
    print(f"{word}: {coef}")

AUC Score: 0.39750908147379344
Top 10 words for Dark skin tone captions:
person: 1.2832798338431541
ocean: 1.0614836026323775
two: 0.9580147531810315
pitch: 0.7918771291789026
room: 0.7654421683933419
racket: 0.7148706918733225
hit: 0.7084709549670664
walking: 0.7074915399237364
motorcycle: 0.6177443185836077
elephants: 0.5842834109440481
Top 10 words for Light skin tone captions:
court: -0.556886159497229
pizza: -0.5586334135162712
parked: -0.6144452905326433
holding: -0.6555308192416662
tie: -0.695505366840133
wii: -0.6988965542068526
hat: -0.7052835469102421
hill: -0.8162199234209225
couple: -0.9740687309113464
next: -1.0842187685658884
